## Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
### * Наименование вакансии.
### * Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
### * Ссылку на саму вакансию.
### * Сайт, откуда собрана вакансия.

In [2]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

In [29]:
def get_dom(vacancy, page=0):
    url_base = 'https://hh.ru/vacancies/'
    
    if vacancy is None:
        vacancy = input('Введите желаемую вакансию: ')

    url = url_base + vacancy

    params = {'page': str(page)}

    headers = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.174 YaBrowser/22.1.3.907 (beta) Yowser/2.5 Safari/537.36'}

    response = requests.get(
        url,
        params=params,
        headers=headers
    )

    dom = bs(response.text, 'html.parser')
    
    return dom

In [59]:
def get_dataframe(dom, website='https://hh.ru'):
    
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item-body__main-info'})
    if vacancies == []:
        return None
    
    df = pd.DataFrame(columns=['name', 'min_salary', 'max_salary', 'currency', 'link', 'website'])
    
    for vacancy in vacancies:
        new_line = []
        
        # Добавление поля name
        name = vacancy.find('a').getText()
        new_line.append(name)
        
        # Добавление полей min_salary и max_salary
        salary = vacancy.find('span', {'class': 'bloko-header-section-3'})
        if salary:
            salary = salary.getText().replace('\u202f', '')
        
        # Среди этих записей часто используется длинное тире, которого нет на обычной раскладке клавиатуры
        # Его номер в таблице ascii 8211б, потому я, также, добавил его сюда
        if salary is None:
            min_salary = np.nan
            max_salary = np.nan
        elif ('от' in salary and 'до' in salary) or '-' in salary or chr(8211) in salary:
            min_salary = re.search(r'\d+', salary)[0]
            max_salary = re.findall(r'\d+', salary)[1]
        elif 'от' in salary:
            min_salary = re.search(r'\d+', salary)[0]
            max_salary = np.nan
        elif 'до' in salary:
            min_salary = np.nan
            max_salary = re.search(r'\d+', salary)[0]
        
        new_line.extend([min_salary, max_salary])
        
        # Добавление поля currency
        if salary:
            currency = re.search(r'[A-zА-я]{3}', salary)[0]
        else:
            currency = np.nan
        
        new_line.append(currency)
        
        # Добавление поля link
        link = vacancy.find('a')['href']
        new_line.append(link)
        
        # Добавление поля website
        new_line.append(website)
        
        df.loc[df.shape[0]] = new_line
                
    return df

In [72]:
df = pd.DataFrame(columns=['name', 'min_salary', 'max_salary', 'currency', 'link', 'website'])

vacancy = input('Введите желаемую вакансию на английском языке: ').replace(' ', '-')

for i in range(40):
    print(f'{i+1} страница')
    dom = get_dom(vacancy, i)
    new_df = get_dataframe(dom)
    
    if new_df is None:
        break
    
    df = pd.concat([df, new_df], ignore_index=True)
    df[['min_price', 'max_price']] = df[['min_price', 'max_price']].astype('float')

    
df

Введите желаемую вакансию на английском языке: dizayner
1 страница
2 страница
3 страница
4 страница
5 страница
6 страница
7 страница
8 страница
9 страница
10 страница
11 страница
12 страница
13 страница
14 страница
15 страница
16 страница
17 страница
18 страница
19 страница
20 страница
21 страница
22 страница
23 страница
24 страница
25 страница
26 страница
27 страница
28 страница
29 страница
30 страница
31 страница
32 страница
33 страница
34 страница
35 страница
36 страница
37 страница
38 страница
39 страница
40 страница


,name,min_salary,max_salary,currency,link,website
0,UX\UI Дизайнер,100000,NaN,руб,https://hh.ru/vacancy/54524294?query=%D0%B4%D0...,https://hh.ru
1,Графический дизайнер,70000,NaN,руб,https://hh.ru/vacancy/53208714?query=%D0%B4%D0...,https://hh.ru
2,Начинающий Web дизайнер,65000,NaN,руб,https://hh.ru/vacancy/53901640?query=%D0%B4%D0...,https://hh.ru
3,Дизайнер,NaN,80000,руб,https://hh.ru/vacancy/54500438?query=%D0%B4%D0...,https://hh.ru
4,UI дизайнер | Motion design | Web design | Бренд,NaN,100000,руб,https://hh.ru/vacancy/54299165?query=%D0%B4%D0...,https://hh.ru
...,...,...,...,...,...,...
1995,Product Owner (remote),NaN,NaN,NaN,https://hh.ru/vacancy/54200842?query=%D0%B4%D0...,https://hh.ru
1996,Продуктовый аналитик (Проект S.Procurement),NaN,NaN,NaN,https://hh.ru/vacancy/54435744?query=%D0%B4%D0...,https://hh.ru
1997,"Lead Software Architect, Database Engine (Java...",550000,NaN,руб,https://hh.ru/vacancy/54002681?query=%D0%B4%D0...,https://hh.ru
1998,Python Architect,NaN,6500,USD,https://hh.ru/vacancy/53034584?query=%D0%B4%D0...,https://hh.ru


In [79]:
df.to_csv('data/vacancy.csv')